# Introduction à  Xarray

Nous allons donner un exemple pratique de l'utilisation du module python **xarray** (aussi désigné par l'abréviation **xr**) qui est une pièce importante du module **xclim**. Pour ce faire nous allons faire une moyenne climatique saisonnière à partir de moyennes mensuelles en utilisant deux méthode:

1. En ne tenant pas compte du nombre de jours dans chaque mois
2. En pondérant les moyennes mensuelles par le nombre de jours dans chaque mois

Nous terminons en comparant le résulat des deux méthodes

## Importation des modules nécessaires

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import xarray as xr

## Ouverture du fichier et extraction des données

on commence par ouvrir un fichier de moyenne mensuelle de température. Ce fichier contient un **xr.Dataset** qui contient plusieurs variables dont **tas**, la température à 2m.




In [ ]:
f_gcm = "<path_to_file>/tas_Amon_CanESM2_historical_r1i1p1_185001-200512.nc"
ds_gcm = xr.open_dataset(f_gcm)
ds_gcm

On extrait ensuite le xr.DataArray de 'tas' du xr.Dataset

In [ ]:
tas_gcm = ds_gcm.tas
tas_gcm

On peut voir que **tas** a plusieurs coordonnées (lon, lat et time) et que la coordonnée time contient un attribut **season**. On peut se servir de cet attribut pour sélectionner les mois d'été (c. à d. **JJA**) seulement. On utilise ensuite l'attribut **time.dt.year** pour faire une sélection des années qui nous intéressent (1950-200 dans cet exemple).

In [ ]:
tas_gcm.time
tas_gcm.time.dt.season

In [ ]:
# on extrait les mois d'été seulement
#
tas_gcm_jja = tas_gcm.sel(time=tas_gcm.time.dt.season == "JJA")

tas_gcm_jja.time.dt.month

In [ ]:
# on selectionne la periode 1950-2000
#
tas_gcm_jja_com = tas_gcm_jja.sel(
    time=(tas_gcm_jja.time.dt.year >= 1950) & (tas_gcm_jja.time.dt.year <= 2000)
)

# raccourci : on pourrait aussi utiliser:
#
# tas_gcm_jja_com = tas_gcm_jja.sel(time=slice('1950','2000'))

tas_gcm_jja_com.time.dt.year

## Calcul de la moyenne saisonnière en utilisant la première méthode

`tas_gcm_jja_com` contient maintenant les moyennes mensuelles pour tous les mois de **JJA** entre 1950 et 2000. On peut donc faire la moyenne saisonnière de l'été en utilisant la première méthode (c. à d. aucune pondération par le nombre de jours dans chaque mois)

In [ ]:
# on calcule la moyenne climatique en utilisant
# le même poids pour chaque mois
#
moy_gcm1 = tas_gcm_jja_com.mean(dim="time")

## Calcul de la moyenne saisonnière en utilisant la 2e méthode

On utilise maintenant la 2e méthode et on tient compte du nombre de jours dans chaque mois. 

On commence par générer un vecteur contenant le nombre de jours dans chaque mois pour une année de 365 jours (le GCM CanESM2 utilisé pour générer les données utilisées utilise un tel calendrier).

In [ ]:
# on calcule le poids de chaques mois selon le nb de jours du mois dans une
# annee non bissextile
#
liste_nb_jours = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
nb_jours = [liste_nb_jours[mois - 1] for mois in tas_gcm_jja_com.time.dt.month.values]
nb_jours

On utilise ensuite ces poids pour pondérer les moyennes mensuelles de chaque mois de `tas_gcm_jja_com`.

In [ ]:
# on calcule la moyenne ponderee par le nombre de jours dans le mois
#
# on ajoute le poids au coordonnees de tas_gcm_jja_com
tas_gcm_jja_com["poids"] = ("time", nb_jours)

# on calcule la somme ponderée des moyennes mensuelles
numerateur = (tas_gcm_jja_com * tas_gcm_jja_com.poids).sum(dim="time")

# on calcule la somme des poids
denominateur = tas_gcm_jja_com.poids.sum(dim="time")

# on calcule la moyenne ponderée
moy_gcm2 = numerateur / denominateur

## Affichage des différences entre les résultats des deux méthodes

On peut maintenant calculer la différences entre les moyennes obtenues en utilisant les deux méthodes. On fait une figure de cette différence et on en affiche les valeurs min et max.

In [ ]:
# difference entre moy_gcm1 et moy_gcm2
#
diff = (moy_gcm1 - moy_gcm2).values
plt.pcolormesh(diff)
plt.colorbar()
plt.title("differences entre les moyennes saisonnières")
plt.show()
print(f"différences min/max = {diff.min():.2f}/{diff.max():.2f} (deg C)")